In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33206
2,Huelva,Confirmados PDIA 14 días,1024
3,Huelva,Tasa PDIA 14 días,"199,54401075666934"
4,Huelva,Confirmados PDIA 7 días,426
5,Huelva,Total Confirmados,33389
6,Huelva,Curados,31407
7,Huelva,Fallecidos,389


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33206.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9091.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 965 personas en los últimos 7 días 

Un positivo PCR cada 358 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33206.0,426.0,1024.0,513170.0,83.013426,199.544011,73.0
Huelva-Costa,19113.0,225.0,574.0,289548.0,77.707323,198.240015,37.0
Condado-Campiña,10810.0,173.0,364.0,156231.0,110.733465,232.988331,32.0
Huelva (capital),9091.0,149.0,402.0,143837.0,103.589480,279.483026,19.0
Cartaya,1394.0,22.0,37.0,20083.0,109.545387,184.235423,10.0
Almonte,1915.0,23.0,56.0,24507.0,93.850737,228.506141,8.0
Moguer,1396.0,33.0,63.0,21867.0,150.912334,288.105364,5.0
San Juan del Puerto,584.0,24.0,51.0,9411.0,255.020720,541.919031,5.0
Sierra de Huelva-Andévalo Central,2930.0,27.0,83.0,67391.0,40.064697,123.161847,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Juan del Puerto,584.0,24.0,51.0,9411.0,255.020720,541.919031,5.0
Puebla de Guzmán,141.0,4.0,15.0,3092.0,129.366106,485.122898,0.0
Beas,232.0,11.0,19.0,4341.0,253.397835,437.687169,0.0
Cerro de Andévalo (El),154.0,6.0,9.0,2327.0,257.842716,386.764074,0.0
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0
Valverde del Camino,663.0,13.0,44.0,12750.0,101.960784,345.098039,3.0
Calañas,201.0,1.0,9.0,2768.0,36.127168,325.144509,0.0
Palos de la Frontera,725.0,20.0,36.0,11742.0,170.328734,306.591722,4.0
Moguer,1396.0,33.0,63.0,21867.0,150.912334,288.105364,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,201.0,1.0,9.0,2768.0,36.127168,325.144509,0.0,0.111111
Niebla,249.0,2.0,9.0,4116.0,48.590865,218.658892,0.0,0.222222
Bollullos Par del Condado,1032.0,6.0,26.0,14387.0,41.704316,180.718704,-1.0,0.230769
Lucena del Puerto,267.0,2.0,8.0,3261.0,61.330880,245.323520,0.0,0.250000
Puebla de Guzmán,141.0,4.0,15.0,3092.0,129.366106,485.122898,0.0,0.266667
Aljaraque,1163.0,6.0,22.0,21474.0,27.940766,102.449474,1.0,0.272727
Aracena,541.0,2.0,7.0,8255.0,24.227741,84.797093,1.0,0.285714
Valverde del Camino,663.0,13.0,44.0,12750.0,101.960784,345.098039,3.0,0.295455
Sierra de Huelva-Andévalo Central,2930.0,27.0,83.0,67391.0,40.064697,123.161847,4.0,0.325301
